In [4]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [5]:
from selenium import webdriver 
from selenium.webdriver.common.by import By


In [2]:
driver= webdriver.Chrome()
driver.get("https://www.nba.com/lakers/team/stats")
stats=driver.find_elements(By.CLASS_NAME, "")



NameError: name 'webdriver' is not defined

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import pandas as pd

# --- Set up Chrome WebDriver ---
options = Options()
# options.add_argument("--headless")  # Uncomment if you want to run in background
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

service = Service()  # Use your local ChromeDriver path if needed
driver = webdriver.Chrome(service=service, options=options)

# --- Load the Lakers stats page ---
url = "https://www.nba.com/lakers/team/stats"
driver.get(url)
time.sleep(8)  # Wait for JavaScript to load the stats

# --- Get the page content after rendering ---
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# --- Find the stats table (robust search) ---
table = soup.find("table")
if table is None:
    raise ValueError("⚠️ Could not find the stats table. Try disabling headless mode or increasing wait time.")

# --- Extract headers ---
thead = table.find("thead")
headers = [th.text.strip() for th in thead.find_all("th")]

# --- Extract player rows ---
tbody = table.find("tbody")
rows = tbody.find_all("tr")
data = []
for row in rows:
    cells = [td.text.strip() for td in row.find_all("td")]
    data.append(cells)

# --- Convert to DataFrame ---
df = pd.DataFrame(data, columns=headers)

# --- Save to CSV ---
df.to_csv("lakers_team_stats.csv", index=False)

# --- Preview ---
df


,Overall,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,...,FT%,OREB,DREB,REB,AST,TO,STL,BLK,PF,+/-
0,2024-25,82,48.1,113.4,40.9,85.5,47.9,13.3,36.4,36.6,...,78.5,9.7,32.8,42.4,26,14,7.7,4.5,17.3,1.2


In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import pandas as pd

# --- Set up Chrome WebDriver ---
options = Options()
# options.add_argument("--headless")  # Uncomment if you want to run in background
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

service = Service()  # Use your local ChromeDriver path if needed
driver = webdriver.Chrome(service=service, options=options)

# --- Load the Lakers stats page ---
url = "https://www.nba.com/lakers/team/stats"
driver.get(url)

# --- Wait for the stats table to load ---
try:
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
except:
    driver.quit()
    raise Exception("⚠️ Table didn't load. Try increasing wait time.")

# --- Get the page content after loading ---
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# --- Find the stats table ---
table = soup.find("table")
if table is None:
    raise ValueError("⚠️ Could not find the stats table.")

# --- Extract headers ---
thead = table.find("thead")
headers = [th.text.strip() for th in thead.find_all("th")]

# --- Extract player rows ---
tbody = table.find("tbody")
rows = tbody.find_all("tr")
data = []
for row in rows:
    cells = [td.text.strip() for td in row.find_all("td")]
    data.append(cells)

# --- Convert to DataFrame ---
df = pd.DataFrame(data, columns=headers)

# --- Save to CSV ---
df.to_csv("lakers_team_stats.csv", index=False)

# --- Preview ---
print(df)


   Overall  GP   MIN    PTS   FGM   FGA   FG%   3PM   3PA   3P%  ...   FT%  \
0  2024-25  82  48.1  113.4  40.9  85.5  47.9  13.3  36.4  36.6  ...  78.5   

  OREB  DREB   REB AST  TO  STL  BLK    PF  +/-  
0  9.7  32.8  42.4  26  14  7.7  4.5  17.3  1.2  

[1 rows x 22 columns]


In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# --- Set up Chrome WebDriver ---
options = Options()
options.add_argument("--headless")  # Run in background
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

service = Service()  # Use your local ChromeDriver path if needed
driver = webdriver.Chrome(service=service, options=options)

# --- Load the Reddit search page ---
url = "https://www.reddit.com/search/?q=lebron&cId=0d114e2f-3066-4ee6-b9e3-50c87fbe4bbc&iId=8b76bc8e-0693-4268-9161-689a76f36474"
driver.get(url)

# --- Wait for the posts to load ---
try:
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.TAG_NAME, "div"))
    )
except:
    driver.quit()
    raise Exception("⚠️ Page didn't load. Try increasing wait time.")

# --- Get the page content after loading ---
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# --- Extract Reddit posts ---
post_titles = []
post_contents = []

# Find all post containers (usually each post is within a div with a specific class name)
posts = soup.find_all("div", class_="Post")
for post in posts:
    title = post.find("h3")  # Title of the post
    if title:
        post_titles.append(title.text.strip())
    
    # Extracting post content (if available)
    content = post.find("div", class_="RichTextJSON-root")
    if content:
        post_contents.append(content.text.strip())

# --- Prepare the data for saving in a text file ---
text_data = "Post Titles and Contents\n"
for title, content in zip(post_titles, post_contents):
    text_data += f"Title: {title}\nContent: {content}\n\n"

# --- Save to text file ---
with open("reddit_lebron_posts.txt", "w", encoding="utf-8") as file:
    file.write(text_data)

# --- Preview ---
print(text_data)


Post Titles and Contents



In [24]:
pip install lxml

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 3.8/3.8 MB 22.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Setup Selenium WebDriver (this will handle the browser automation)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# URL of the player's stats page
url = "https://www.nba.com/stats/player/2544?SeasonType=Regular+Season&PerMode=Totals"
driver.get(url)

# Wait for the page to fully load (you might need to adjust this)
driver.implicitly_wait(10)

# Now, extract the page source after it has been fully loaded
page_source = driver.page_source

# You can use BeautifulSoup to parse this page source
from bs4 import BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find all tables in the page
tables = soup.find_all('table')

# Loop through the tables and convert them to pandas DataFrames
dfs = []
for table in tables:
    # Convert each table into a pandas DataFrame
    df = pd.read_html(str(table))[0]
    dfs.append(df)

# Display the first table (or adjust based on the table you want)
print(dfs[0])

# Close the browser window
driver.quit()


C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\4094904899.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\4094904899.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\4094904899.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\4094904899.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

   Traditional Splits                      Unnamed: 5_level_0  \
              By Year TEAM  GP   MIN   PTS                FGM   
0             2024-25  LAL  70  2444  1710                651   
1             2023-24  LAL  71  2504  1822                685   
2             2022-23  LAL  55  1954  1590                609   
3             2021-22  LAL  56  2084  1695                640   
4             2020-21  LAL  45  1504  1126                422   
5             2019-20  LAL  67  2316  1698                643   
6             2018-19  LAL  55  1937  1505                558   
7             2017-18  CLE  82  3026  2251                857   
8             2016-17  CLE  74  2795  1954                736   
9             2015-16  CLE  76  2709  1920                737   
10            2014-15  CLE  69  2493  1743                624   
11            2013-14  MIA  77  2902  2089                767   
12            2012-13  MIA  76  2877  2036                765   
13            2011-12  MI

In [28]:
# dfs

In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Setup Selenium WebDriver (this will handle the browser automation)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# URL of the player's stats page
url = "https://www.nba.com/stats/player/2544?SeasonType=Regular+Season&PerMode=Totals"
driver.get(url)

# Wait for the page to fully load (you might need to adjust this)
driver.implicitly_wait(10)

# Now, extract the page source after it has been fully loaded
page_source = driver.page_source

# You can use BeautifulSoup to parse this page source
from bs4 import BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find all tables in the page
tables = soup.find_all('table')

# Loop through the tables and convert them to pandas DataFrames
dfs = []
for table in tables:
    # Convert each table into a pandas DataFrame
    df = pd.read_html(str(table))[0]
    dfs.append(df)

# Choose the first DataFrame (or modify this if you need a different table)
#df = dfs[0]

# Save the DataFrame to a CSV file
df.to_csv('lebron.csv', index=False)

# Display the DataFrame (optional)
print(df)

# Close the browser window
driver.quit()


C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\5516012.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\5516012.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\5516012.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\5516012.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal

   Usage Splits                      Unnamed: 5_level_0 Unnamed: 6_level_0  \
        By Year TEAM  GP   MIN  USG%               %FGM               %FGA   
0       2024-25  LAL  70  2444  29.1               30.9               29.2   
1       2023-24  LAL  71  2504  28.5               29.4               28.0   
2       2022-23  LAL  55  1954  32.2               33.4               33.4   
3       2021-22  LAL  56  2084  31.7               35.2               32.5   
4       2020-21  LAL  45  1504  31.0               32.2               30.8   
5       2019-20  LAL  67  2316  30.8               30.9               30.5   
6       2018-19  LAL  55  1937  31.1               31.6               30.5   
7       2017-18  CLE  82  3026  31.0               33.1               29.6   
8       2016-17  CLE  74  2795  29.2               30.6               27.2   
9       2015-16  CLE  76  2709  30.5               32.7               29.9   
10      2014-15  CLE  69  2493  31.8               30.9         

### Curry CSV

In [38]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Setup Selenium WebDriver (this will handle the browser automation)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# URL of the player's stats page
url = "https://www.nba.com/stats/player/201939"
driver.get(url)

# Wait for the page to fully load (you might need to adjust this)
driver.implicitly_wait(10)

# Now, extract the page source after it has been fully loaded
page_source = driver.page_source

# You can use BeautifulSoup to parse this page source
from bs4 import BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find all tables in the page
tables = soup.find_all('table')

# Loop through the tables and convert them to pandas DataFrames
dfs = []
for table in tables:
    # Convert each table into a pandas DataFrame
    df = pd.read_html(str(table))[0]
    dfs.append(df)

# Choose the first DataFrame (or modify this if you need a different table)
#df = dfs[0]

# Save the DataFrame to a CSV file
df.to_csv('curry.csv', index=False)

# Display the DataFrame (optional)
print(df)

# Close the browser window
driver.quit()

C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\311695429.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\311695429.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\311695429.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\311695429.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a

   Usage Splits                      Unnamed: 5_level_0 Unnamed: 6_level_0  \
        By Year TEAM  GP   MIN  USG%               %FGM               %FGA   
0       2024-25  GSW  70  2252  28.6               27.6               29.2   
1       2023-24  GSW  74  2422  30.2               29.2               31.4   
2       2022-23  GSW  56  1941  29.7               30.3               30.4   
3       2021-22  GSW  64  2211  29.3               27.4               30.2   
4       2020-21  GSW  63  2152  33.1               33.9               34.0   
5       2019-20  GSW   5   139  28.6               26.2               27.8   
6       2018-19  GSW  69  2331  29.2               28.3               30.4   
7       2017-18  GSW  51  1631  29.0               27.3               29.0   
8       2016-17  GSW  79  2639  28.6               27.0               29.3   
9       2015-16  GSW  79  2700  31.4               32.5               32.4   
10      2014-15  GSW  80  2613  27.7               27.4         

### KD CSV

In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Setup Selenium WebDriver (this will handle the browser automation)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# URL of the player's stats page
url = "https://www.nba.com/stats/player/201142"
driver.get(url)

# Wait for the page to fully load (you might need to adjust this)
driver.implicitly_wait(10)

# Now, extract the page source after it has been fully loaded
page_source = driver.page_source

# You can use BeautifulSoup to parse this page source
from bs4 import BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find all tables in the page
tables = soup.find_all('table')

# Loop through the tables and convert them to pandas DataFrames
dfs = []
for table in tables:
    # Convert each table into a pandas DataFrame
    df = pd.read_html(str(table))[0]
    dfs.append(df)

# Choose the first DataFrame (or modify this if you need a different table)
#df = dfs[0]

# Save the DataFrame to a CSV file
df.to_csv('kevin durant.csv', index=False)

# Display the DataFrame (optional)
print(df)

# Close the browser window
driver.quit()

C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\3177765408.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\3177765408.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\3177765408.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\3177765408.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

   Usage Splits                      Unnamed: 5_level_0 Unnamed: 6_level_0  \
        By Year TEAM  GP   MIN  USG%               %FGM               %FGA   
0       2024-25  PHX  62  2265  28.3               29.8               28.4   
1       2023-24  PHX  75  2791  28.3               29.7               28.7   
2       2022-23  TOT  47  1672  30.5               31.9               29.7   
3       2022-23  PHX   8   269  27.3               30.8               27.5   
4       2022-23  BKN  39  1403  31.1               32.1               30.1   
5       2021-22  BKN  55  2047  31.0               31.1               30.2   
6       2020-21  BKN  35  1157  30.3               29.5               28.9   
7       2018-19  GSW  78  2702  28.3               28.3               27.6   
8       2017-18  GSW  68  2325  29.3               29.5               29.4   
9       2016-17  GSW  62  2070  27.1               28.6               27.3   
10      2015-16  OKC  72  2578  29.8               30.8         

## Giannis CSV


In [40]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Setup Selenium WebDriver (this will handle the browser automation)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# URL of the player's stats page
url = "https://www.nba.com/stats/player/203507"
driver.get(url)

# Wait for the page to fully load (you might need to adjust this)
driver.implicitly_wait(10)

# Now, extract the page source after it has been fully loaded
page_source = driver.page_source

# You can use BeautifulSoup to parse this page source
from bs4 import BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find all tables in the page
tables = soup.find_all('table')

# Loop through the tables and convert them to pandas DataFrames
dfs = []
for table in tables:
    # Convert each table into a pandas DataFrame
    df = pd.read_html(str(table))[0]
    dfs.append(df)

# Choose the first DataFrame (or modify this if you need a different table)
#df = dfs[0]

# Save the DataFrame to a CSV file
df.to_csv('giannis.csv', index=False)

# Display the DataFrame (optional)
print(df)

# Close the browser window
driver.quit()

C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\3304029617.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\3304029617.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\3304029617.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\3304029617.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

  Usage Splits                     Unnamed: 5_level_0 Unnamed: 6_level_0  \
       By Year TEAM  GP  MIN  USG%               %FGM               %FGA   
0      2024-25  MIL   4  144  35.9               42.5               31.9   
1      2022-23  MIL   3   91  34.2               37.3               32.1   
2      2021-22  MIL  12  448  37.3               38.0               34.8   
3      2020-21  MIL  21  800  31.4               35.9               29.3   
4      2019-20  MIL   9  277  33.9               37.5               31.4   
5      2018-19  MIL  15  514  31.0               31.1               28.0   
6      2017-18  MIL   7  280  27.6               30.7               27.1   
7      2016-17  MIL   6  243  29.8               33.5               28.2   
8      2014-15  MIL   6  201  20.4               18.6               19.3   

  Unnamed: 7_level_0 Unnamed: 8_level_0 Unnamed: 9_level_0  ...  \
                %3PM               %3PA               %FTM  ...   
0                0.0         

### The code with the function


In [37]:
# # scrape_multi_players.py

# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager
# from bs4 import BeautifulSoup
# import pandas as pd
# import time
# import re

# # --- Set up Chrome WebDriver ---
# options = Options()
# # options.add_argument("--headless")      # Uncomment to run in background
# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")

# service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service, options=options)

# # --- List of player-stats URLs ---
# players = [
#     "https://www.nba.com/stats/player/2544?SeasonType=Regular+Season&PerMode=Totals",
#     "https://www.nba.com/stats/player/201939?PerMode=Totals",
#     "https://www.nba.com/stats/player/977/career",
#     "https://www.nba.com/stats/player/203999?SeasonType=Regular+Season&PerMode=Totals",
#     "https://www.nba.com/stats/player/406/career?PerMode=Totals",
#     "https://www.nba.com/stats/player/201566?SeasonType=Regular+Season&PerMode=Totals",
#     "https://www.nba.com/stats/player/203507?SeasonType=Regular+Season&PerMode=Totals"
# ]

# # Will hold your DataFrames if you need them after the loop
# all_dfs = {}

# for url in players:
#     driver.get(url)
#     # give JS time to render the table
#     time.sleep(8)

#     # pull in the rendered DOM
#     soup = BeautifulSoup(driver.page_source, "html.parser")

#     # find the first <table> on the page
#     table = soup.find("table")
#     if table is None:
#         print(f"No table found at {url}")
#         continue

#     # pandas will parse out the HTML table for us
#     df = pd.read_html(str(table))[0]

#     # extract the player ID from the URL for naming
#     name = driver.find_element(By.CLASS_NAME, "PlayerSummary_playerNameText___MhqC").text


#     # save to CSV
#     csv_name = f"player_{name}_stats.csv"
#     df.to_csv(csv_name, index=False)
#     print(f"✔️ Saved main table for player {name} to {csv_name}")

#     # stash the DataFrame if you need it later
#     all_dfs[pid] = df

# driver.quit()

# # optional: do something with all_dfs, e.g. display keys
# print("Done. DataFrames created for:", list(all_dfs.keys()))


Data Analysis

In [42]:
df_giannis=pd.read_csv("giannis.csv")
df_giannis

,Usage Splits,Usage Splits.1,Usage Splits.2,Usage Splits.3,Usage Splits.4,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0,...,Usage Splits.25,Usage Splits.26,Usage Splits.27,Usage Splits.28,Usage Splits.29,Usage Splits.30,Usage Splits.31,Usage Splits.32,Usage Splits.33,Usage Splits.34
0,By Year,TEAM,GP,MIN,USG%,%FGM,%FGA,%3PM,%3PA,%FTM,...,Unnamed: 50_level_1,Unnamed: 51_level_1,Unnamed: 52_level_1,Unnamed: 53_level_1,Unnamed: 54_level_1,Unnamed: 55_level_1,Unnamed: 56_level_1,Unnamed: 57_level_1,Unnamed: 58_level_1,Unnamed: 59_level_1
1,2024-25,MIL,4,144,35.9,42.5,31.9,0.0,2.9,58.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-23,MIL,3,91,34.2,37.3,32.1,0.0,4.8,32.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-22,MIL,12,448,37.3,38.0,34.8,8.2,13.1,59.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-21,MIL,21,800,31.4,35.9,29.3,7.3,12.1,48.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2019-20,MIL,9,277,33.9,37.5,31.4,15.9,17.2,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2018-19,MIL,15,514,31.0,31.1,28.0,13.1,13.3,46.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017-18,MIL,7,280,27.6,30.7,27.1,7.4,9.9,44.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2016-17,MIL,6,243,29.8,33.5,28.2,8.7,9.0,34.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2014-15,MIL,6,201,20.4,18.6,19.3,0.0,1.6,30.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
df_cleaned_giannis=df_giannis.iloc[:,:10]
df_cleaned_giannis

,Usage Splits,Usage Splits.1,Usage Splits.2,Usage Splits.3,Usage Splits.4,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0
0,By Year,TEAM,GP,MIN,USG%,%FGM,%FGA,%3PM,%3PA,%FTM
1,2024-25,MIL,4,144,35.9,42.5,31.9,0.0,2.9,58.9
2,2022-23,MIL,3,91,34.2,37.3,32.1,0.0,4.8,32.6
3,2021-22,MIL,12,448,37.3,38.0,34.8,8.2,13.1,59.1
4,2020-21,MIL,21,800,31.4,35.9,29.3,7.3,12.1,48.6
5,2019-20,MIL,9,277,33.9,37.5,31.4,15.9,17.2,50.0
6,2018-19,MIL,15,514,31.0,31.1,28.0,13.1,13.3,46.1
7,2017-18,MIL,7,280,27.6,30.7,27.1,7.4,9.9,44.2
8,2016-17,MIL,6,243,29.8,33.5,28.2,8.7,9.0,34.2
9,2014-15,MIL,6,201,20.4,18.6,19.3,0.0,1.6,30.4


In [49]:
df_lebron=pd.read_csv('lebron.csv')
df_cleaned_lebron=df_lebron.iloc[:,:10]
df_cleaned_giannis

,Usage Splits,Usage Splits.1,Usage Splits.2,Usage Splits.3,Usage Splits.4,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0
0,By Year,TEAM,GP,MIN,USG%,%FGM,%FGA,%3PM,%3PA,%FTM
1,2024-25,MIL,4,144,35.9,42.5,31.9,0.0,2.9,58.9
2,2022-23,MIL,3,91,34.2,37.3,32.1,0.0,4.8,32.6
3,2021-22,MIL,12,448,37.3,38.0,34.8,8.2,13.1,59.1
4,2020-21,MIL,21,800,31.4,35.9,29.3,7.3,12.1,48.6
5,2019-20,MIL,9,277,33.9,37.5,31.4,15.9,17.2,50.0
6,2018-19,MIL,15,514,31.0,31.1,28.0,13.1,13.3,46.1
7,2017-18,MIL,7,280,27.6,30.7,27.1,7.4,9.9,44.2
8,2016-17,MIL,6,243,29.8,33.5,28.2,8.7,9.0,34.2
9,2014-15,MIL,6,201,20.4,18.6,19.3,0.0,1.6,30.4


In [51]:
df_curry=pd.read_csv('curry.csv')
df_cleaned_curry=df_curry.iloc[:,:10]
df_cleaned_curry

,Usage Splits,Usage Splits.1,Usage Splits.2,Usage Splits.3,Usage Splits.4,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0
0,By Year,TEAM,GP,MIN,USG%,%FGM,%FGA,%3PM,%3PA,%FTM
1,2024-25,GSW,70,2252,28.6,27.6,29.2,40.8,38.9,34.3
2,2023-24,GSW,74,2422,30.2,29.2,31.4,45.7,43.8,36.2
3,2022-23,GSW,56,1941,29.7,30.3,30.4,38.7,36.2,41.4
4,2021-22,GSW,64,2211,29.3,27.4,30.2,40.0,39.4,39.5
5,2020-21,GSW,63,2152,33.1,33.9,34.0,48.1,43.9,44.4
6,2019-20,GSW,5,139,28.6,26.2,27.8,31.6,38.6,46.4
7,2018-19,GSW,69,2331,29.2,28.3,30.4,47.9,44.1,31.6
8,2017-18,GSW,51,1631,29.0,27.3,29.0,41.9,42.1,41.2
9,2016-17,GSW,79,2639,28.6,27.0,29.3,41.8,41.0,30.8


In [54]:
df_kd=pd.read_csv("kevin durant.csv")
df_cleaned_kd=df_kd.iloc[:,:10]
df_cleaned_kd

,Usage Splits,Usage Splits.1,Usage Splits.2,Usage Splits.3,Usage Splits.4,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0
0,By Year,TEAM,GP,MIN,USG%,%FGM,%FGA,%3PM,%3PA,%FTM
1,2024-25,PHX,62,2265,28.3,29.8,28.4,24.3,22.8,36.4
2,2023-24,PHX,75,2791,28.3,29.7,28.7,22.6,21.8,33.2
3,2022-23,TOT,47,1672,30.5,31.9,29.7,22.1,21.9,50.7
4,2022-23,PHX,8,269,27.3,30.8,27.5,33.3,24.4,36.0
5,2022-23,BKN,39,1403,31.1,32.1,30.1,20.1,21.5,54.0
6,2021-22,BKN,55,2047,31.0,31.1,30.2,23.6,23.2,46.3
7,2020-21,BKN,35,1157,30.3,29.5,28.9,23.8,22.7,44.2
8,2018-19,GSW,78,2702,28.3,28.3,27.6,17.7,19.8,43.0
9,2017-18,GSW,68,2325,29.3,29.5,29.4,29.0,27.9,41.1


In [70]:
df_cleaned_lebron

,Usage Splits,Usage Splits.1,Usage Splits.2,Usage Splits.3,Usage Splits.4,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0
0,By Year,TEAM,GP,MIN,USG%,%FGM,%FGA,%3PM,%3PA,%FTM
1,2024-25,LAL,70,2444,29.1,30.9,29.2,22.6,21.4,28.6
2,2023-24,LAL,71,2504,28.5,29.4,28.0,24.3,22.7,31.9
3,2022-23,LAL,55,1954,32.2,33.4,33.4,26.7,29.2,33.0
4,2021-22,LAL,56,2084,31.7,35.2,32.5,29.6,29.5,37.2
5,2020-21,LAL,45,1504,31.0,32.2,30.8,28.3,28.0,32.6
6,2019-20,LAL,67,2316,30.8,30.9,30.5,26.5,27.1,33.4
7,2018-19,LAL,55,1937,31.1,31.6,30.5,27.8,27.1,40.3
8,2017-18,CLE,82,3026,31.0,33.1,29.6,18.6,19.4,34.1
9,2016-17,CLE,74,2795,29.2,30.6,27.2,15.8,16.9,35.5


In [74]:
df_cleaned_lebron['Usage Splits.3'] = pd.to_numeric(df_cleaned_lebron['Usage Splits.3'], errors='coerce')
mean_value = df_cleaned_lebron['Usage Splits.3'].mean()
print(mean_value)

2683.7727272727275


C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\3435090115.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned_lebron['Usage Splits.3'] = pd.to_numeric(df_cleaned_lebron['Usage Splits.3'], errors='coerce')


In [76]:
df_cleaned_kd['Usage Splits.3'] = pd.to_numeric(df_cleaned_kd['Usage Splits.3'], errors='coerce')
mean_value_kd = df_cleaned_kd['Usage Splits.3'].mean()
print(mean_value_kd)

2258.3684210526317


C:\Users\Jacob\AppData\Local\Temp\ipykernel_27468\1698934918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned_kd['Usage Splits.3'] = pd.to_numeric(df_cleaned_kd['Usage Splits.3'], errors='coerce')
